In [ ]:
import tensorflow as tf

# Ensure TensorFlow is running in eager execution mode at the very beginning
tf.compat.v1.enable_eager_execution()

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import json
import pickle
import random
import logging

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')

# Set TensorFlow logging level to INFO
tf.get_logger().setLevel(logging.INFO)

# Load data from the intents JSON file
with open("intents.json") as file:
    data = json.load(file)

# Try to load preprocessed data from a pickle file
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except FileNotFoundError:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    # Tokenize and stem the words in the patterns
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    # Create the bag of words model
    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            bag.append(1 if w in wrds else 0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)

    # Save the preprocessed data to a pickle file
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

# Ensure TensorFlow is running in eager execution mode
print("Eager execution enabled:", tf.executing_eagerly())

# Define the TensorFlow model
model = Sequential([
    Dense(8, input_shape=(len(training[0]),), activation='relu'),
    Dense(8, activation='relu'),
    Dense(len(output[0]), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Flag to force retraining
force_retrain = False

if force_retrain:
    print("Forcing retraining of the model...")
    model.fit(training, output, epochs=1000, batch_size=8, verbose=1)
    model.save("model.h5")
    print("Model retrained and saved as 'model.h5'.")
else:
    try:
        model.load_weights("model.h5")
        print("Model weights loaded successfully.")
    except OSError:
        print("Model weights not found, starting training...")
        # Train the model if weights are not found
        model.fit(training, output, epochs=1000, batch_size=8, verbose=1)
        model.save("model.h5")
        print("Model training complete and saved as 'model.h5'.")

# Function to convert user input into bag of words
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

# Function to start the chat
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(np.array([bag_of_words(inp, words)]))
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

# Start the chat
chat()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/viktoriiadrozdovska/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/viktoriiadrozdovska/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Eager execution enabled: True
Model weights loaded successfully.
Start talking with the bot (type quit to stop)!
